# Downloading data

In [18]:
import torch
import yfinance as yf

# Other files
import utils
from models.autowarp import AutoWarp
from models.mean_variance_optimisation import MeanVarianceOptimisation
from models.autoencoders import LinearAutoencoder, ConvAutoencoder, ConvLinearAutoEncoder, train_autoencoder
from models.fama_french import FamaFrench

# General imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta


# Optimisation
from pypfopt.expected_returns import mean_historical_return
from pypfopt.hierarchical_portfolio import HRPOpt

# Set seed for reproducibility
torch.manual_seed(0)
np.random.seed(0);

In [19]:
# Plotting adjustments
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['savefig.dpi'] = 300
plt.rcParams.update({'figure.autolayout': True})

In [20]:
# 2 years training (split into 2 year train and 1 year valid), 18 months test
start_date = '2018-09-01'
valid_date = '2020-09-01'
train_date = '2021-09-01'
end_date   = '2023-03-01'

# Open json file with parameters
with open(f'params/sp500_{start_date}.json') as f:
    params = json.load(f)

In [21]:
# Download data
# data_dict = {'Linear': {}, 'CNN': {}, 'Linear + CNN': {}}
# for model in ['Linear', 'CNN', 'Linear + CNN']:
#     data_arrays, price_dfs = utils.split_data(start_date, valid_date, train_date, end_date, train_valid_split=2 / 3, **params[model]['sharpe']['data'])
#     data_dict[model]['data_train'] = data_arrays[0]
#     assert np.isnan(data_arrays[0]).sum() == 0

data_arrays, price_dfs = utils.split_data(start_date=start_date, valid_date=valid_date, train_date=train_date, end_date=end_date, train_valid_split=2 / 3, returns=False, momentum=False)

prices_train = price_dfs[0]
prices_test = price_dfs[3]
data_train = data_arrays[0]

num_epochs=20

In [22]:
# S&P 500
sp500 = yf.download("^GSPC", start=train_date, end=end_date, period="1d", progress=False)['Adj Close']
sp500 = sp500.div(sp500.iloc[0]).mul(100)

# Calculating autoencoder distance matrix

In [23]:
models = {}
models['Linear'] = train_autoencoder(LinearAutoencoder, input_size=data_train.shape[1], num_epochs=num_epochs, data=data_train, verbose=True, **params['Linear']['sharpe']['autoencoder'])
models['CNN'] = train_autoencoder(ConvAutoencoder, input_size=data_train.shape[1], num_epochs=num_epochs, data=data_train, verbose=True, **params['CNN']['sharpe']['autoencoder'])
models['Linear + CNN'] =train_autoencoder(ConvLinearAutoEncoder, input_size=data_train.shape[1], num_epochs=num_epochs, data=data_train, verbose=True, **params['Linear + CNN']['sharpe']['autoencoder'])

LinearAutoencoder
Epoch [1/20], Loss: 0.0635
Epoch [2/20], Loss: 0.0132
Epoch [3/20], Loss: 0.0093
Epoch [4/20], Loss: 0.0079
Epoch [5/20], Loss: 0.0072
Epoch [6/20], Loss: 0.0068
Epoch [7/20], Loss: 0.0063
Epoch [8/20], Loss: 0.0056
Epoch [9/20], Loss: 0.0052
Epoch [10/20], Loss: 0.0048
Epoch [11/20], Loss: 0.0044
Epoch [12/20], Loss: 0.0042
Epoch [13/20], Loss: 0.0041
Epoch [14/20], Loss: 0.0039
Epoch [15/20], Loss: 0.0037
Epoch [16/20], Loss: 0.0035
Epoch [17/20], Loss: 0.0034
Epoch [18/20], Loss: 0.0032
Epoch [19/20], Loss: 0.0031
Epoch [20/20], Loss: 0.0030
ConvAutoencoder
Epoch [1/20], Loss: 0.1258
Epoch [2/20], Loss: 0.0665
Epoch [3/20], Loss: 0.0349
Epoch [4/20], Loss: 0.0229
Epoch [5/20], Loss: 0.0190
Epoch [6/20], Loss: 0.0168
Epoch [7/20], Loss: 0.0150
Epoch [8/20], Loss: 0.0138
Epoch [9/20], Loss: 0.0130
Epoch [10/20], Loss: 0.0124
Epoch [11/20], Loss: 0.0119
Epoch [12/20], Loss: 0.0115
Epoch [13/20], Loss: 0.0111
Epoch [14/20], Loss: 0.0107
Epoch [15/20], Loss: 0.0105
Epoc

In [24]:
%%time
dist_matrices = {}
for model in ['Linear', 'Linear + CNN', 'CNN']:
    print(model)
    learner = AutoWarp(models[model], data_train, **params[model]['sharpe']['dist_matrix'])
    learner.learn_metric(verbose=True)
    dist_matrices[model] = learner.create_distance_matrix()

Linear
Alpha: 0.4150773882865906
Gamma: 0.22196239233016968
Epsilon: 0.7845350503921509
Linear + CNN


TypeError: unsupported operand type(s) for -: 'ValueError' and 'ValueError'

# Mean variance optimisation

In [25]:
# Empty dict for weights
weights = {}

for model in ["Linear", "CNN", "Linear + CNN"]:

    # Setup
    e_returns = mean_historical_return(prices_train)
    optimiser = MeanVarianceOptimisation(expected_returns = e_returns, prices = prices_train, solver='ECOS', weight_bounds = (0,1))

    # Get weights
    risk_matrix = optimiser.make_risk_matrix(dist_matrices[model], **params[model]['sharpe']['risk_matrix'])
    weights[model], train_sr = optimiser.max_sharpe_ratio(risk_matrix=risk_matrix, l2_reg=0)

    # Print out summary
    print(model, "train SR:", train_sr)

for model in ["Covariance", "Covariance Shrinkage", "EW Covariance"]:

    # Setup
    e_returns = mean_historical_return(prices_train)
    optimiser = MeanVarianceOptimisation(expected_returns = e_returns, prices = prices_train, solver='ECOS', weight_bounds = (0,1))

    # Get weights
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        risk_matrix = optimiser.benchmark_matrix(model)
        weights[model], train_sr = optimiser.max_sharpe_ratio(risk_matrix=risk_matrix, l2_reg=0)

    # Print out summary
    print(model, "train SR:", train_sr)

# Factor model
e_returns = mean_historical_return(prices_train)
optimiser = MeanVarianceOptimisation(expected_returns = e_returns, prices = prices_train, solver='ECOS', weight_bounds = (0,1))
fama_french = FamaFrench(prices_train, file_path='data/F-F_Research_Data_Factors_daily.CSV', n_rows=25419)
fama_cov = fama_french.get_covariance_matrix()
weights['Fama-French 3 factor'], train_sr = optimiser.max_sharpe_ratio(risk_matrix=fama_cov, l2_reg=0)
print("Fama-French 3 factor train SR:", train_sr)

# HRP
rets = (prices_train/prices_train.shift(1)-1).dropna()
hrp = HRPOpt(rets)
hrp.optimize()
weights['HRP'] = pd.DataFrame.from_dict(hrp.clean_weights(), orient='index', columns=['weights'])
train_sr = utils.calculate_sharpe_ratio(prices = prices_test, weights = weights['HRP'])
print("HRP train SR:", train_sr)

Linear train SR: 1.7469835067284967


KeyError: 'CNN'

# Evaulating performance on the test set

In [ ]:
# Plots
for model in ["Linear", "CNN", "Linear + CNN", "Covariance", "Covariance Shrinkage", "EW Covariance", "Fama-French 3 factor", "HRP"]:
    plt.plot(prices_test @ weights[model], label=model)
plt.plot(sp500, label="S&P 500")
plt.plot(prices_test, alpha=0.05)
plt.ylim(50, 200)
plt.grid()
plt.gcf().autofmt_xdate()
plt.legend(ncol=2);

In [ ]:
# Sharpe ratio for each model
for model in ["Linear", "CNN", "Linear + CNN", "Covariance", "Covariance Shrinkage", "EW Covariance", "Fama-French 3 factor", "HRP"]:
    print(model, "sharpe",  utils.calculate_sharpe_ratio(prices = prices_test, weights = weights[model]))
print("S&P 500:", utils.calculate_sp500_sharpe(train_date, end_date))

In [ ]:
for model in ["Linear", "CNN", "Linear + CNN", "Covariance", "Covariance Shrinkage", "EW Covariance", "Fama-French 3 factor", "HRP"]:
    print(model, "max drawdown:", utils.calculate_max_drawdown(prices_test, weights[model])*100)
print("S&P 500:", utils.calculate_sp500_drawdown(train_date, end_date))

In [ ]:
for model in ["Linear", "CNN", "Linear + CNN", "Covariance", "Covariance Shrinkage", "EW Covariance", "Fama-French 3 factor", "HRP"]:
    print(model, "number of stocks included:", np.count_nonzero(weights[model]))

# Plotting all results

In [ ]:
results = {}
for start_date in ["2017-03-01", "2017-09-01", "2018-03-01", "2018-09-01"]:
    results[start_date] = {}
    for opt in ['sharpe', 'volatility']:
   # Open json file with parameters
        with open(f'results/sp500_{start_date}_{opt}.json') as f:
            results[start_date][opt] = json.load(f)['sharpe_ratios']

In [ ]:
# convert dict of dicts to dataframe
results_df = pd.DataFrame.from_dict({(i,j): results[i][j]
                           for i in results.keys()
                           for j in results[i].keys()},
                       orient='index')

In [ ]:
# Seperate dataframe for when second index is 'sharpe' and 'volatility'
results_sharpe = results_df[results_df.index.get_level_values(1) == 'sharpe'].droplevel(1)
results_volatility = results_df[results_df.index.get_level_values(1) == 'volatility'].droplevel(1)

In [ ]:
colours = ['indianred', 'brown', 'maroon', 'slateblue', 'mediumslateblue', 'darkslateblue', 'mediumseagreen', 'gold', 'gray']

fig, ax = plt.subplots(2, 2, figsize=(4, 6))
i=0
for date in results_sharpe.index:

    # Calculate test period
    test_start = datetime.strptime(date, '%Y-%m-%d') + relativedelta(years=3)
    test_end = test_start + relativedelta(years=1, months=6)

    # Plotting
    ax.flatten()[i].bar(results_sharpe[results_sharpe.index==date].columns, np.squeeze(results_sharpe[results_sharpe.index==date].values), color = colours)
    plt.setp(ax.flatten()[i].get_xticklabels(), rotation=45, ha='right', fontsize=8)
    plt.setp(ax.flatten()[i].get_yticklabels(), fontsize=8)
    min_y = np.squeeze(results_sharpe[results_sharpe.index==date].values).min()
    max_y = np.squeeze(results_sharpe[results_sharpe.index==date].values).max()
    if min_y<0 and max_y<0:
        max_y-=0.1
        min_y -= 0.1
    elif min_y>0 and max_y>0:
        min_y=0
    ax.flatten()[i].set_ylim(top = max_y+0.1, bottom = min_y)
    ax.flatten()[i].set_title(f"{test_start.strftime('%Y-%m-%d')} - {test_end.strftime('%Y-%m-%d')}", fontsize = 8)

    # Add values to bars
    for j, v in enumerate(np.squeeze(results_sharpe[results_sharpe.index==date].values)):
        if v > 0:
            ax.flatten()[i].text(j, v+0.02, "{:.2f}".format(v), color='black', fontsize=6, ha='center')
        else:
            ax.flatten()[i].text(j, v-0.03, "{:.2f}".format(v), color='black', fontsize=6, ha='center')

    i+=1

ax[0,0].set_ylabel('Sharpe ratio', fontsize=9)
ax[1,0].set_ylabel('Sharpe ratio', fontsize=9)

fig.suptitle('Maximising the Sharpe Ratio', fontsize=10)
fig.tight_layout()

fig.savefig('plots/max_sharpe_results.png')

In [ ]:
colours = ['indianred', 'brown', 'maroon', 'slateblue', 'mediumslateblue', 'darkslateblue', 'mediumseagreen', 'gold', 'gray']

fig, ax = plt.subplots(2, 2, figsize=(4, 6))
i=0
for date in results_volatility.index:

    # Calculate test period
    test_start = datetime.strptime(date, '%Y-%m-%d') + relativedelta(years=3)
    test_end = test_start + relativedelta(years=1, months=6)

    # Plotting
    ax.flatten()[i].bar(results_volatility[results_volatility.index==date].columns, np.squeeze(results_volatility[results_volatility.index==date].values), color = colours)
    plt.setp(ax.flatten()[i].get_xticklabels(), rotation=45, ha='right', fontsize=8)
    plt.setp(ax.flatten()[i].get_yticklabels(), fontsize=8)
    min_y = np.squeeze(results_volatility[results_volatility.index==date].values).min()
    max_y = np.squeeze(results_volatility[results_volatility.index==date].values).max()
    if (min_y<0) and (max_y<0):
        max_y-=0.1
        min_y -= 0.1
    elif (min_y>0) and (max_y>0):
        min_y=0
    else:
        min_y-=0.1
    ax.flatten()[i].set_ylim(top = max_y+0.1, bottom = min_y)
    ax.flatten()[i].set_title(f"{test_start.strftime('%Y-%m-%d')} - {test_end.strftime('%Y-%m-%d')}", fontsize = 8)

    # Add values to bars
    for j, v in enumerate(np.squeeze(results_volatility[results_volatility.index==date].values)):
        if v > 0:
            ax.flatten()[i].text(j, v+0.02, "{:.2f}".format(v), color='black', fontsize=6, ha='center')
        else:
            ax.flatten()[i].text(j, v-0.03, "{:.2f}".format(v), color='black', fontsize=6, ha='center')

    i+=1

ax[0,0].set_ylabel('Sharpe ratio', fontsize=9)
ax[1,0].set_ylabel('Sharpe ratio', fontsize=9)

fig.suptitle('Minimising Volatility', fontsize=10)
fig.tight_layout()

fig.savefig('plots/min_vol_results.png');